## MongoDB Atlas

To use MongoDB Atlas, you must first deploy a cluster. To get started head over to Atlas here: [quick start](https://www.mongodb.com/docs/atlas/getting-started/).
Create an Atlas database and create an Atlas Search Index to search vectors.

Follow below MongoDB Atlas guide
- [Create new cluster](https://www.mongodb.com/docs/atlas/tutorial/create-new-cluster/)
- [Connect to database](https://www.mongodb.com/docs/atlas/driver-connection/)
- [Create an Atlas Vector Search Index](https://www.mongodb.com/docs/atlas/atlas-vector-search/create-index/)
- [Create Index Fields](https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#std-label-avs-types-vector-search)

### benefits?
- use mongoDB itself!

In [34]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25 pymongo langchain langchain-mongodb


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [35]:

%load_ext dotenv
%dotenv
# UPSTAGE_API_KEY
# ... KEY

The dotenv module is not an IPython extension.


UsageError: Line magic function `%dotenv` not found.


In [2]:
import warnings

warnings.filterwarnings("ignore")

In [37]:
from pymongo.mongo_client import MongoClient
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_upstage import UpstageEmbeddings
import os

"""
Your connection string should use following format:
mongodb+srv://<username>:<password>@<clusterName>.<hostname>.mongodb.net
"""
MONGO_URI = os.environ.get("MONGO_URI")

# Connect to your Atlas cluster
client = MongoClient(MONGO_URI)
# Define collection and index name
db_name = "db"
collection_name = "collection"
atlas_collection = client[db_name][collection_name]
vector_search_index = "vector_index"

# Create Indexes

create atlas index fields.

[Create Index Fields](https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#std-label-avs-types-vector-search)

```json
{
  "fields": [
    {
      "numDimensions": 4096,
      "path": "embedding",
      "similarity": "dotProduct",
      "type": "vector"
    }
  ]
}
```

In [38]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


sample_text = [
    "Korea is a beautiful country to visit in the spring.",
    "The best time to visit Korea is in the fall.",
    "Best way to find bug is using unit test.",
    "Python is a great programming language for beginners.",
    "Sung Kim is a great teacher.",
]

splits = RecursiveCharacterTextSplitter().create_documents(sample_text)

print(splits)

vectorstore = MongoDBAtlasVectorSearch.from_documents(
    documents=splits,
    collection=atlas_collection,
    embedding=UpstageEmbeddings(model="solar-embedding-1-large"),
    index_name=vector_search_index
)

[Document(page_content='Korea is a beautiful country to visit in the spring.'), Document(page_content='The best time to visit Korea is in the fall.'), Document(page_content='Best way to find bug is using unit test.'), Document(page_content='Python is a great programming language for beginners.'), Document(page_content='Sung Kim is a great teacher.')]


In [39]:
# check if text is in the vector store
def is_in_vectorstore(collection, text):
    # we can query to mongodb collection
    result = collection.find_one({ "text": text })
    return result is not None

In [40]:
is_in_vectorstore(atlas_collection, "Hello, new sentence")

False

In [41]:
is_in_vectorstore(atlas_collection, splits[0].page_content)

True

In [42]:
from langchain_upstage import UpstageLayoutAnalysisLoader


layzer = UpstageLayoutAnalysisLoader("pdfs/kim-tse-2008.pdf", output_type="html")
# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
docs = layzer.load()  # or layzer.lazy_load()

In [43]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

# 2. Split
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=100, language=Language.HTML
)
splits = text_splitter.split_documents(docs)
print("Splits:", len(splits))

Splits: 125


In [44]:
from langchain_mongodb import MongoDBAtlasVectorSearch

vectorstore = MongoDBAtlasVectorSearch(
    collection=atlas_collection,
    embedding=UpstageEmbeddings(model="solar-embedding-1-large"),
    index_name=vector_search_index
)
retriever = vectorstore.as_retriever()


unique_splits = [
    split for split in splits if not is_in_vectorstore(atlas_collection, split.page_content)
]
print(len(unique_splits))

# 3. Embed & indexing if it's not in the vector store
if len(unique_splits) > 0:
    MongoDBAtlasVectorSearch.from_documents(
    documents=unique_splits,
    collection=atlas_collection,
    embedding=UpstageEmbeddings(model="solar-embedding-1-large"),
    index_name=vector_search_index
)

125


In [49]:
# Query the retriever
search_result = retriever.invoke("How to find problems in code?")
print(search_result)
print(search_result[0].page_content[:100])


[Document(page_content='Best way to find bug is using unit test.', metadata={'_id': '64435a9d-c192-58eb-1b75-296b8426d51f'}), Document(page_content='Best way to find bug is using unit test.', metadata={'_id': {'$oid': '664e8aeac5ca68643b344c50'}}), Document(page_content="<p id='13' style='font-size:16px'>introduced bugs immediately. Several bug-finding techni-<br>ques could be used, including code inspections, unit testing,<br>and the use of static analysis tools. Since these steps would<br>be taken right after a code change was made, the developer<br>would still retain the full mental context of the change. This<br>holds promise for reducing the time required to find<br>software bugs and reducing the time that bugs stay resident<br>in software before removal.</p><br>", metadata={'_id': {'$oid': '664e8b2ec5ca68643b344c5a'}, 'total_pages': 16, 'type': 'html', 'split': 'none'}), Document(page_content="<p id='39' style='font-size:20px'>2.2 Mining Buggy Patterns</p><br><p id='40' style='fo